In [1]:
%pip install -U opensearch-py==2.3.1
%pip install -U boto3==1.33.2
%pip install -U retrying==1.3.4
!pip install langchain
!pip install langchain-community
!pip install streamlit
!pip install PyPDF2
!pip install tqdm
!pip install opensearch-py
!pip install requests-aws4auth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.3/327.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 29.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.1 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 110.4 MB/s eta 0:00:0000:01:0

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import boto3
import random
import time


suffix = random.randrange(200, 900)
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
iam_client = boto3_session.client('iam')
account_number = boto3.client('sts').get_caller_identity().get('Account')
identity = boto3.client('sts').get_caller_identity()['Arn']

encryption_policy_name = f"bedrock-sample-rag-sp-{suffix}"
network_policy_name = f"bedrock-sample-rag-np-{suffix}"
access_policy_name = f'bedrock-sample-rag-ap-{suffix}'
bedrock_execution_role_name = f'AmazonBedrockExecutionRoleForKnowledgeBase_{suffix}'
fm_policy_name = f'AmazonBedrockFoundationModelPolicyForKnowledgeBase_{suffix}'
s3_policy_name = f'AmazonBedrockS3PolicyForKnowledgeBase_{suffix}'
oss_policy_name = f'AmazonBedrockOSSPolicyForKnowledgeBase_{suffix}'


def create_bedrock_execution_role(bucket_name):
    foundation_model_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                ],
                "Resource": [
                    f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"
                ]
            }
        ]
    }

    s3_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject",
                    "s3:ListBucket"
                ],
                "Resource": [
                    f"arn:aws:s3:::{bucket_name}",
                    f"arn:aws:s3:::{bucket_name}/*"
                ],
                "Condition": {
                    "StringEquals": {
                        "aws:ResourceAccount": f"{account_number}"
                    }
                }
            }
        ]
    }

    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    # create policies based on the policy documents
    fm_policy = iam_client.create_policy(
        PolicyName=fm_policy_name,
        PolicyDocument=json.dumps(foundation_model_policy_document),
        Description='Policy for accessing foundation model',
    )

    s3_policy = iam_client.create_policy(
        PolicyName=s3_policy_name,
        PolicyDocument=json.dumps(s3_policy_document),
        Description='Policy for reading documents from s3')

    # create bedrock execution role
    bedrock_kb_execution_role = iam_client.create_role(
        RoleName=bedrock_execution_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_document),
        Description='Amazon Bedrock Knowledge Base Execution Role for accessing OSS and S3',
        MaxSessionDuration=3600
    )

    # fetch arn of the policies and role created above
    bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']
    s3_policy_arn = s3_policy["Policy"]["Arn"]
    fm_policy_arn = fm_policy["Policy"]["Arn"]

    # attach policies to Amazon Bedrock execution role
    iam_client.attach_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyArn=fm_policy_arn
    )
    iam_client.attach_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyArn=s3_policy_arn
    )
    return bedrock_kb_execution_role


def create_oss_policy_attach_bedrock_execution_role(collection_id, bedrock_kb_execution_role):
    # define oss policy document
    oss_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "aoss:APIAccessAll"
                ],
                "Resource": [
                    f"arn:aws:aoss:{region_name}:{account_number}:collection/{collection_id}"
                ]
            }
        ]
    }
    oss_policy = iam_client.create_policy(
        PolicyName=oss_policy_name,
        PolicyDocument=json.dumps(oss_policy_document),
        Description='Policy for accessing opensearch serverless',
    )
    oss_policy_arn = oss_policy["Policy"]["Arn"]
    print("Opensearch serverless arn: ", oss_policy_arn)

    iam_client.attach_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyArn=oss_policy_arn
    )
    return None


def create_policies_in_oss(vector_store_name, aoss_client, bedrock_kb_execution_role_arn):
    encryption_policy = aoss_client.create_security_policy(
        name=encryption_policy_name,
        policy=json.dumps(
            {
                'Rules': [{'Resource': ['collection/' + vector_store_name],
                           'ResourceType': 'collection'}],
                'AWSOwnedKey': True
            }),
        type='encryption'
    )

    network_policy = aoss_client.create_security_policy(
        name=network_policy_name,
        policy=json.dumps(
            [
                {'Rules': [{'Resource': ['collection/' + vector_store_name],
                            'ResourceType': 'collection'}],
                 'AllowFromPublic': True}
            ]),
        type='network'
    )
    access_policy = aoss_client.create_access_policy(
        name=access_policy_name,
        policy=json.dumps(
            [
                {
                    'Rules': [
                        {
                            'Resource': ['collection/' + vector_store_name],
                            'Permission': [
                                'aoss:CreateCollectionItems',
                                'aoss:DeleteCollectionItems',
                                'aoss:UpdateCollectionItems',
                                'aoss:DescribeCollectionItems'],
                            'ResourceType': 'collection'
                        },
                        {
                            'Resource': ['index/' + vector_store_name + '/*'],
                            'Permission': [
                                'aoss:CreateIndex',
                                'aoss:DeleteIndex',
                                'aoss:UpdateIndex',
                                'aoss:DescribeIndex',
                                'aoss:ReadDocument',
                                'aoss:WriteDocument'],
                            'ResourceType': 'index'
                        }],
                    'Principal': [identity, bedrock_kb_execution_role_arn],
                    'Description': 'Easy data policy'}
            ]),
        type='data'
    )
    return encryption_policy, network_policy, access_policy


def delete_iam_role_and_policies():
    fm_policy_arn = f"arn:aws:iam::{account_number}:policy/{fm_policy_name}"
    s3_policy_arn = f"arn:aws:iam::{account_number}:policy/{s3_policy_name}"
    oss_policy_arn = f"arn:aws:iam::{account_number}:policy/{oss_policy_name}"
    iam_client.detach_role_policy(
        RoleName=bedrock_execution_role_name,
        PolicyArn=s3_policy_arn
    )
    iam_client.detach_role_policy(
        RoleName=bedrock_execution_role_name,
        PolicyArn=fm_policy_arn
    )
    iam_client.detach_role_policy(
        RoleName=bedrock_execution_role_name,
        PolicyArn=oss_policy_arn
    )
    iam_client.delete_role(RoleName=bedrock_execution_role_name)
    iam_client.delete_policy(PolicyArn=s3_policy_arn)
    iam_client.delete_policy(PolicyArn=fm_policy_arn)
    iam_client.delete_policy(PolicyArn=oss_policy_arn)
    return 0


def interactive_sleep(seconds: int):
    dots = ''
    for i in range(seconds):
        dots += '.'
        print(dots, end='\r')
        time.sleep(1)
    # print('Done!')

In [2]:
import json
import os
import boto3
from botocore.exceptions import ClientError
import pprint
#from utils.utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, interactive_sleep
import random
from retrying import retry

In [4]:
suffix = random.randrange(200, 900)

sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
service = 'aoss'
s3_client = boto3.client('s3')
account_id = sts_client.get_caller_identity()["Account"]
identifier= "new_chunk"
s3_suffix = f"{region_name}-{account_id}-{identifier}"
bucket_name = f'bedrock-kb-{s3_suffix}' # replace it with your bucket name.
pp = pprint.PrettyPrinter(indent=2)

In [5]:
# Check if bucket exists, and if not create S3 bucket for knowledge base data source
try:
    s3_client.head_bucket(Bucket=bucket_name)
    print(f'Bucket {bucket_name} Exists')
except ClientError as e:
    print(f'Creating bucket {bucket_name}')
    s3bucket = s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={ 'LocationConstraint': region_name }
    )

Creating bucket bedrock-kb-us-west-2-734389953405


In [6]:
import boto3
import time
vector_store_name = f'bedrock-sample-rag-{suffix}'
index_name = f"bedrock-sample-rag-index-{suffix}"
aoss_client = boto3_session.client('opensearchserverless')
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']

In [7]:
# create security, network and data access policies within OSS
encryption_policy, network_policy, access_policy = create_policies_in_oss(vector_store_name=vector_store_name,
                       aoss_client=aoss_client,
                       bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn)
collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

In [8]:
pp.pprint(collection)

{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '314',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Wed, 05 Jun 2024 09:37:16 '
                                                 'GMT',
                                         'x-amzn-requestid': 'eaa243c5-1242-40ab-a953-77a0d460be4d'},
                        'HTTPStatusCode': 200,
                        'RequestId': 'eaa243c5-1242-40ab-a953-77a0d460be4d',
                        'RetryAttempts': 0},
  'createCollectionDetail': { 'arn': 'arn:aws:aoss:us-west-2:734389953405:collection/qd75qkwby8mzb7wbocoh',
                              'createdDate': 1717580235597,
                              'id': 'qd75qkwby8mzb7wbocoh',
                              'kmsKeyArn': 'auto',
                              'lastModifiedDate': 1717580235597,
                             

In [9]:
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

qd75qkwby8mzb7wbocoh.us-west-2.aoss.amazonaws.com


In [10]:
response = aoss_client.batch_get_collection(names=[vector_store_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    interactive_sleep(30)
    response = aoss_client.batch_get_collection(names=[vector_store_name])
print('\nCollection successfully created:')
pp.pprint(response["collectionDetails"])

Creating collection...
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
Creating collection...........
..............................
Collection successfully created:
[ { 'arn': 'arn:aws:aoss:us-west-2:734389953405:collection/qd75qkwby8mzb7wbocoh',
    'collectionEndpoint': 'https://qd75qkwby8mzb7wbocoh.us-west-2.aoss.amazonaws.com',
    'createdDate': 1717580235597,
    'dashboardEndpoint': 'https://qd75qkwby8mzb7wbocoh.us-west-2.aoss.amazonaws.com/_dashboards',
    'id': 'qd75qkwby8mzb7wbocoh',
    'kmsKeyArn': 'auto',
    'lastModifiedDate': 1717580623662,
    'name': 'bedrock-sample-rag-697',
    'standbyReplicas': 'ENABLED',
    'status': 'ACTIVE',
    'type': 'VECTORSEARCH'}]


In [11]:
try:
    create_oss_policy_attach_bedrock_execution_role(collection_id=collection_id,
                                                    bedrock_kb_execution_role=bedrock_kb_execution_role)
    # It can take up to a minute for data access rules to be enforced
    interactive_sleep(60)
except Exception as e:
    print("Policy already exists")
    pp.pprint(e)

Opensearch serverless arn:  arn:aws:iam::734389953405:policy/AmazonBedrockOSSPolicyForKnowledgeBase_795


# for fixed size chunking

In [12]:
# Create the vector index in Opensearch serverless, with the knn_vector field index mapping, specifying the dimension size, name and engine.
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestError
credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

index_name = f"bedrock-sample-index-{suffix}"
body_json = {
   "settings": {
      "index.knn": "true",
       "number_of_shards": 1,
       "knn.algo_param.ef_search": 512,
       "number_of_replicas": 0,
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1536,
             "method": {
                 "name": "hnsw",
                 "engine": "faiss",
                 "space_type": "l2"
             },
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         }
      }
   }
}

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

In [13]:
try:
    response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
    print('\nCreating index:')
    pp.pprint(response)

    # index creation can take up to a minute
    interactive_sleep(60)
except RequestError as e:
    # you can delete the index if its already exists
    # oss_client.indices.delete(index=index_name)
    print(f'Error while trying to create the index, with error {e.error}\nyou may unmark the delete above to delete, and recreate the index')
    



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-index-697',
  'shards_acknowledged': True}


In [14]:
opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]['arn'],
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

# Ingest strategy - How to ingest data from the data source
chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# The data source to ingest documents from, into the OpenSearch serverless knowledge base index
s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
    # "inclusionPrefixes":["*.*"] # you can use this if you want to create a KB using data within s3 prefixes.
}

# The embedding model used by Bedrock to embed ingested documents, and realtime prompts
embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"

name = f"bedrock-sample-knowledge-base-{suffix}"
description = "Amazon Q documentation"
roleArn = bedrock_kb_execution_role_arn

In [15]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = name,
        description = description,
        roleArn = roleArn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [16]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

In [17]:
pp.pprint(kb)

{ 'createdAt': datetime.datetime(2024, 6, 5, 9, 45, 56, 373735, tzinfo=tzlocal()),
  'description': 'Amazon Q documentation',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:734389953405:knowledge-base/8SYD79ZBQV',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v1'}},
  'knowledgeBaseId': '8SYD79ZBQV',
  'name': 'bedrock-sample-knowledge-base-697',
  'roleArn': 'arn:aws:iam::734389953405:role/AmazonBedrockExecutionRoleForKnowledgeBase_795',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 'arn:aws:aoss:us-west-2:734389953405:collection/qd75qkwby8mzb7wbocoh',
                                                                   'fieldMapping': { 'metadataField': 'text-metadata',
                                                                                     'tex

In [18]:
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

In [19]:
# Create a DataSource in KnowledgeBase 
create_ds_response = bedrock_agent_client.create_data_source(
    name = name,
    description = description,
    knowledgeBaseId = kb['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
ds = create_ds_response["dataSource"]
pp.pprint(ds)

{ 'createdAt': datetime.datetime(2024, 6, 5, 9, 45, 57, 4514, tzinfo=tzlocal()),
  'dataSourceConfiguration': { 's3Configuration': { 'bucketArn': 'arn:aws:s3:::bedrock-kb-us-west-2-734389953405'},
                               'type': 'S3'},
  'dataSourceId': 'UNNDUF4NNU',
  'description': 'Amazon Q documentation',
  'knowledgeBaseId': '8SYD79ZBQV',
  'name': 'bedrock-sample-knowledge-base-697',
  'status': 'AVAILABLE',
  'updatedAt': datetime.datetime(2024, 6, 5, 9, 45, 57, 4514, tzinfo=tzlocal()),
  'vectorIngestionConfiguration': { 'chunkingConfiguration': { 'chunkingStrategy': 'FIXED_SIZE',
                                                               'fixedSizeChunkingConfiguration': { 'maxTokens': 512,
                                                                                                   'overlapPercentage': 20}}}}


In [20]:
bedrock_agent_client.get_data_source(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

{'ResponseMetadata': {'RequestId': 'ec3faa63-a1cd-4f4d-8663-216a82a9248a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 05 Jun 2024 09:45:57 GMT',
   'content-type': 'application/json',
   'content-length': '584',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ec3faa63-a1cd-4f4d-8663-216a82a9248a',
   'x-amz-apigw-id': 'Y40JVEQavHcEcxQ=',
   'x-amzn-trace-id': 'Root=1-666033d5-61370d095e9e56763de76e5e'},
  'RetryAttempts': 0},
 'dataSource': {'knowledgeBaseId': '8SYD79ZBQV',
  'dataSourceId': 'UNNDUF4NNU',
  'name': 'bedrock-sample-knowledge-base-697',
  'status': 'AVAILABLE',
  'description': 'Amazon Q documentation',
  'dataSourceConfiguration': {'type': 'S3',
   's3Configuration': {'bucketArn': 'arn:aws:s3:::bedrock-kb-us-west-2-734389953405'}},
  'vectorIngestionConfiguration': {'chunkingConfiguration': {'chunkingStrategy': 'FIXED_SIZE',
    'fixedSizeChunkingConfiguration': {'maxTokens': 512,
     'overlapPercentage': 20}}},
  'createdAt': datetime.datetime(202

In [22]:
# Start an ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [23]:
job = start_job_response["ingestionJob"]
pp.pprint(job)

{ 'dataSourceId': 'UNNDUF4NNU',
  'ingestionJobId': 'OUEFAKGSIX',
  'knowledgeBaseId': '8SYD79ZBQV',
  'startedAt': datetime.datetime(2024, 6, 5, 9, 46, 44, 167238, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2024, 6, 5, 9, 46, 44, 167238, tzinfo=tzlocal())}


In [24]:
# Get job 
while(job['status']!='COMPLETE' ):
    get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb['knowledgeBaseId'],
        dataSourceId = ds["dataSourceId"],
        ingestionJobId = job["ingestionJobId"]
  )
    job = get_job_response["ingestionJob"]
    
    interactive_sleep(30)

pp.pprint(job)

{ 'dataSourceId': 'UNNDUF4NNU',
  'ingestionJobId': 'OUEFAKGSIX',
  'knowledgeBaseId': '8SYD79ZBQV',
  'startedAt': datetime.datetime(2024, 6, 5, 9, 46, 44, 167238, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 1},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2024, 6, 5, 9, 46, 49, 107853, tzinfo=tzlocal())}


In [25]:
kb_id_1 = kb["knowledgeBaseId"]
pp.pprint(kb_id)

'8SYD79ZBQV'


In [26]:
# keep the kb_id for invocation later in the invoke request
%store kb_id

Stored 'kb_id' (str)


In [27]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-west-2


In [28]:
def retrieve(query, kbId, numberOfResults):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults
            }
        }
    )


In [42]:
def retrieve_information(query, num_results):
    response = retrieve(query, kb_id, num_results)
    retrievalResults = response['retrievalResults']
    pp = pprint.PrettyPrinter(indent=2)
    return retrievalResults

In [43]:
query = "What are the key features of amazon q developer?"
retrievalResults=retrieve_information(query,2)

In [44]:
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [45]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

['Amazon Q Developer User Guide What is Amazon Q Developer? Note Powered by '
 'Amazon Bedrock: AWS implements automated abuse detection. Because Amazon Q '
 'Developer is built on Amazon Bedrock, users can take full advantage of the '
 'controls implemented in Amazon Bedrock to enforce safety, security, and the '
 'responsible use of artificial intelligence (AI). Amazon Q Developer is a '
 'generative artificial intelligence (AI) powered conversational assistant '
 'that can help you understand, build, extend, and operate AWS applications. '
 'You can ask questions about AWS architecture, your AWS resources, best '
 'practices, documentation, support, and more. Amazon Q is constantly updating '
 'its capabilities so your questions get the most contextually relevant and '
 'actionable answers. When used in an integrated development environment '
 '(IDE), Amazon Q provides software development assistance. Amazon Q can chat '
 'about code, provide inline code completions, generate net ne

# chunking

In [68]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Initialize S3 client
s3_client = boto3.client('s3')

# Define your S3 bucket and file names
bucket_name = 'gen-ai-dummy'
input_file_name = 'extracted_text.txt'
output_prefix = 'chunks/'

# Download the extracted text file from S3
s3_client.download_file(bucket_name, input_file_name, '/tmp/extracted_text.txt')

# Read the extracted text
with open('/tmp/extracted_text.txt', 'r') as file:
    extracted_text = file.read()

# Initialize the text splitter with chunk size of 500 and overlap of 150
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150
)

# Split the text into chunks
chunks = text_splitter.split_text(extracted_text)

In [69]:
chunks

['Amazon Q Developer\nUser Guide\nWhat is Amazon Q Developer?\nNote\nPowered by Amazon Bedrock: AWS implements automated abuse detection. Because\nAmazon Q Developer is built on Amazon Bedrock, users can take full advantage of the\ncontrols implemented in Amazon Bedrock to enforce safety, security, and the responsible\nuse of artificial intelligence (AI).\nAmazon Q Developer is a generative artificial intelligence (AI) powered conversational assistant',
 'use of artificial intelligence (AI).\nAmazon Q Developer is a generative artificial intelligence (AI) powered conversational assistant\nthat can help you understand, build, extend, and operate AWS applications. You can ask questions\nabout AWS architecture, your AWS resources, best practices, documentation, support, and more.\nAmazon Q is constantly updating its capabilities so your questions get the most contextually\nrelevant and actionable answers.',
 'Amazon Q is constantly updating its capabilities so your questions get the most 

In [70]:
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')

# Define the S3 bucket name
bucket_name = 'bedrock-kb-us-west-2-734389953405-newchunk'



# Loop through the chunks and create text files
for i, chunk in enumerate(chunks, start=1):
    file_name = f'chunk_{i}.txt'
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=chunk)
    print(f'Uploaded {file_name} to {bucket_name}')

print("All chunks have been uploaded to S3.")


Uploaded chunk_1.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_2.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_3.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_4.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_5.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_6.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_7.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_8.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_9.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_10.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_11.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_12.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_13.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_14.txt to bedrock-kb-us-west-2-734389953405-newchunk
Uploaded chunk_15.txt to bedrock-kb-us-west-2-73438995340

# none chunking application

In [80]:
kb_id_1 = "YELOBDEE2V"
pp.pprint(kb_id_1)

'YELOBDEE2V'


In [81]:
# keep the kb_id for invocation later in the invoke request
%store kb_id_1

Stored 'kb_id_1' (str)


In [82]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-west-2


In [88]:
def retrieve(query, kbId_1, numberOfResults):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId_1,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults
            }
        }
    )


In [89]:
def retrieve_information(query, num_results):
    response = retrieve(query, kb_id_1, num_results)
    retrievalResults = response['retrievalResults']
    pp = pprint.PrettyPrinter(indent=10)
    return retrievalResults

In [105]:
query = "how to set up amazon q dev?"
retrievalResults=retrieve_information(query,5)

In [106]:
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [107]:
contexts

['Amazon Q Developer User Guide What is Amazon Q Developer? Note Powered by Amazon Bedrock: AWS implements automated abuse detection. Because Amazon Q Developer is built on Amazon Bedrock, users can take full advantage of the controls implemented in Amazon Bedrock to enforce safety, security, and the responsible use of artificial intelligence (AI). Amazon Q Developer is a generative artificial intelligence (AI) powered conversational assistant that can help you understand, build, extend, and operate AWS applications. You can ask questions about AWS architecture, your AWS resources, best practices, documentation, support, and more. Amazon Q is constantly updating its capabilities so your questions get the most contextually relevant and actionable answers. When used in an integrated development environment (IDE), Amazon Q provides software development assistance. Amazon Q can chat about code, provide inline code completions, generate net new code, scan your code for security vulnerabilit

# RAG Fusion

## none chunking

In [233]:
kb_id_1 = "YELOBDEE2V"
pp.pprint(kb_id_1)

'YELOBDEE2V'


In [234]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-west-2


In [235]:
def retrieve(query, kbId_1, numberOfResults):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId_1,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults
            }
        }
    )


In [236]:
def retrieve_information(query, num_results):
    response = retrieve(query, kb_id_1, num_results)
    retrievalResults = response['retrievalResults']
    pp = pprint.PrettyPrinter(indent=10)
    return retrievalResults

In [237]:
query = "what are Key features of Amazon Q Developer?"
retrievalResults_2=retrieve_information(query,10)

In [238]:
retrievalResults_2

[{'content': {'text': 'Amazon Q Developer\nUser Guide\nWhat is Amazon Q Developer?\nNote\nPowered by Amazon Bedrock: AWS implements automated abuse detection. Because\nAmazon Q Developer is built on Amazon Bedrock, users can take full advantage of the\ncontrols implemented in Amazon Bedrock to enforce safety, security, and the responsible\nuse of artificial intelligence (AI).\nAmazon Q Developer is a generative artificial intelligence (AI) powered conversational assistant'},
  'location': {'type': 'S3',
   's3Location': {'uri': 's3://gen-ai-dummy/chunk_1.txt'}},
  'score': 0.66296},
 {'content': {'text': 'use of artificial intelligence (AI).\nAmazon Q Developer is a generative artificial intelligence (AI) powered conversational assistant\nthat can help you understand, build, extend, and operate AWS applications. You can ask questions\nabout AWS architecture, your AWS resources, best practices, documentation, support, and more.\nAmazon Q is constantly updating its capabilities so your q

## for fixed chunking

In [239]:
kb_id = "8SYD79ZBQV"
pp.pprint(kb_id)

'8SYD79ZBQV'


In [240]:
# keep the kb_id for invocation later in the invoke request
%store kb_id

Stored 'kb_id' (str)


In [241]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-west-2


In [242]:
def retrieve(query, kbId, numberOfResults):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults
            }
        }
    )

In [243]:
def retrieve_information(query, num_results):
    response = retrieve(query, kb_id, num_results)
    retrievalResults = response['retrievalResults']
    pp = pprint.PrettyPrinter(indent=2)
    return retrievalResults

In [244]:
query = "what are Key features of Amazon Q Developer?"
retrievalResults_1=retrieve_information(query,10)

In [245]:
retrievalResults_1

[{'content': {'text': 'Amazon Q Developer User Guide What is Amazon Q Developer? Note Powered by Amazon Bedrock: AWS implements automated abuse detection. Because Amazon Q Developer is built on Amazon Bedrock, users can take full advantage of the controls implemented in Amazon Bedrock to enforce safety, security, and the responsible use of artificial intelligence (AI). Amazon Q Developer is a generative artificial intelligence (AI) powered conversational assistant that can help you understand, build, extend, and operate AWS applications. You can ask questions about AWS architecture, your AWS resources, best practices, documentation, support, and more. Amazon Q is constantly updating its capabilities so your questions get the most contextually relevant and actionable answers. When used in an integrated development environment (IDE), Amazon Q provides software development assistance. Amazon Q can chat about code, provide inline code completions, generate net new code, scan your code for 

## RAG fusion

In [246]:
def reciprocal_rank_fusion(*ranked_lists, k=60):
    scores = {}
    for ranked_list in ranked_lists:
        for rank, result in enumerate(ranked_list, start=1):
            doc = result['content']['text']
            scores[doc] = scores.get(doc, 0) + 1 / (rank + k)
    sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [doc for doc, score in sorted_docs]


In [247]:
fused_results = reciprocal_rank_fusion(retrievalResults_1, retrievalResults_2)

In [248]:
def augment_query(query, documents):
    fused_context = " ".join(documents)
    augmented_query = f"{query} Context: {fused_context}"
    return augmented_query

In [249]:
augmented_query = augment_query(query, fused_results)

In [250]:
augmented_query

"what are Key features of Amazon Q Developer? Context: Amazon Q Developer User Guide What is Amazon Q Developer? Note Powered by Amazon Bedrock: AWS implements automated abuse detection. Because Amazon Q Developer is built on Amazon Bedrock, users can take full advantage of the controls implemented in Amazon Bedrock to enforce safety, security, and the responsible use of artificial intelligence (AI). Amazon Q Developer is a generative artificial intelligence (AI) powered conversational assistant that can help you understand, build, extend, and operate AWS applications. You can ask questions about AWS architecture, your AWS resources, best practices, documentation, support, and more. Amazon Q is constantly updating its capabilities so your questions get the most contextually relevant and actionable answers. When used in an integrated development environment (IDE), Amazon Q provides software development assistance. Amazon Q can chat about code, provide inline code completions, generate n

# LLM call

In [171]:
prompt = f"""
Human: You are a machine learning engineer AI system, providing answers to questions using fact-based and statistical information when possible. Use the following pieces of documentation to provide a concise answer to the question enclosed in <question> tags. If you don't know the answer, just say that you don't know, and do not attempt to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

In [172]:
# payload with model paramters
messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(augmented_query, query)}]}]
sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2000,
    "messages": messages,
    "temperature": 0.5,
    "top_p": 1
        }  )


In [173]:
modelId = 'anthropic.claude-3-opus-20240229-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']

pp.pprint(response_text)

('Yes, Amazon Q can scan your code for security vulnerabilities. According to '
 'the provided documentation, within IDEs, Amazon Q scans your code for '
 'security vulnerabilities and code quality issues. You can get security '
 'recommendations as you code or scan entire projects to monitor the security '
 'posture of your applications throughout development. For more information, '
 'the documentation suggests seeing the section "Scan your code with Amazon '
 'Q."')


In [149]:
modelId = 'anthropic.claude-3-opus-20240229-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']

pp.pprint(response_text)

('Yes, you can scan your code for security vulnerabilities using Amazon Q. '
 'According to the documentation:\n'
 '\n'
 '"Within IDEs, Amazon Q scans your code for security vulnerabilities and code '
 'quality issues. You can get security recommendations as you code or scan '
 'entire projects to monitor the security posture of your applications '
 'throughout development."\n'
 '\n'
 'So Amazon Q provides real-time security scanning as you write code in your '
 'IDE, as well as the ability to scan entire projects to identify '
 'vulnerabilities. This allows you to proactively find and fix security issues '
 'during the development process.')


# Agent 

In [252]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.chat_models import BedrockChat
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
import boto3

# Initialize Bedrock client and model
bedrock = boto3.client(service_name="bedrock-runtime")
llm = BedrockChat(model_id="anthropic.claude-3-opus-20240229-v1:0", client=bedrock, model_kwargs={"max_tokens": 1000})

# Define a prompt template for the whole chain
combined_prompt = PromptTemplate(input_variables=["combined_input"], template="""
Query and Context: {combined_input}
Step 1: Based on the provided context, identify the main topic. If the context is insufficient, state 'I don't know'.
Step 2: Generate a detailed explanation for the identified topic. If the context is insufficient, state 'I don't know'.
Step 3: Summarize the key points from the detailed explanation. If the context is insufficient, state 'I don't know'.
""")

# Create a single chain that handles all steps
chain = LLMChain(llm=llm, prompt=combined_prompt)

# Function to run the chain of thought reasoning
def run_chain_of_thought(query, context):
    combined_input = f"Query: {query}. Context: {context}"
    result = chain.run({"combined_input": combined_input})
    return result

# Example usage
query = query
context =augmented_query

# Run chain of thought
result = run_chain_of_thought(query, context)
print(result)


Based on the provided Amazon Q Developer User Guide excerpt, the main topic is the key features of Amazon Q Developer.

The key features of Amazon Q Developer include:

1. Chat - Amazon Q can answer natural language questions about AWS services, best practices, documentation, the AWS CLI, and more. It provides summaries, instructions, and links to information sources.

2. Conversation memory - Amazon Q uses the context of the current conversation to inform future responses during that conversation. 

3. Code improvements and advice - Within IDEs, Amazon Q can answer software development questions, improve existing code, and generate new code.

4. Code completion - Amazon Q provides real-time code suggestions based on the developer's current code and comments as they write code in the IDE.

5. Troubleshooting and support - Amazon Q can help diagnose errors in the AWS Management Console and connect users with live AWS Support agents to address questions and issues.

In summary, the key f

In [265]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.chat_models import BedrockChat
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import boto3

# Initialize Bedrock client and model
bedrock = boto3.client(service_name="bedrock-runtime")
llm = BedrockChat(model_id="anthropic.claude-3-opus-20240229-v1:0", client=bedrock, model_kwargs={"max_tokens": 1000})

# Define generic prompt templates for different branches
identify_topic_prompt = PromptTemplate(
    input_variables=["context"], 
    template="Step 1: Based on the provided context, identify the main topic. If the context is insufficient, state 'I don't know'.\nContext: {context}"
)

detailed_explanation_prompt = PromptTemplate(
    input_variables=["topic", "context"], 
    template="Step 2a: Provide a detailed explanation for the identified topic: {topic}. Context: {context}. If the context is insufficient, state 'I don't know'."
)

polished_language_prompt = PromptTemplate(
    input_variables=["topic", "context"], 
    template="Step 2b: Provide a polished explanation for the identified topic: {topic}. Context: {context}. If the context is insufficient, state 'I don't know'."
)

summarize_prompt = PromptTemplate(
    input_variables=["detail", "context"], 
    template="Step 3: Summarize the key points from the detail provided: {detail}. Context: {context}. If the context is insufficient, state 'I don't know'."
)

evaluate_prompt = PromptTemplate(
    input_variables=["query", "context", "result1", "result2"],
    template="Given the query '{query}' and context '{context}', evaluate which of the two results is better:\nResult 1: {result1}\nResult 2: {result2}\nSelect the better result. Do not select 'I don't know' responses."
)

# Create chains for each step
identify_topic_chain = LLMChain(llm=llm, prompt=identify_topic_prompt)
detailed_explanation_chain = LLMChain(llm=llm, prompt=detailed_explanation_prompt)
polished_language_chain = LLMChain(llm=llm, prompt=polished_language_prompt)
summarize_chain = LLMChain(llm=llm, prompt=summarize_prompt)
evaluate_chain = LLMChain(llm=llm, prompt=evaluate_prompt)

# Function to evaluate results
def evaluate_results(query, context, result1, result2):
    #print(f"Evaluating results: \nResult 1: {result1}\nResult 2: {result2}\n")
    evaluation_result = evaluate_chain.run({
        "query": query,
        "context": context,
        "result1": result1,
        "result2": result2
    })
    #print(f"Evaluation result: {evaluation_result}\n")
    return evaluation_result

# Function to run the tree of thought reasoning
def run_tree_of_thought(query, context):
    # Step 1: Identify the main topic
    topic_result = identify_topic_chain.run({"context": context})
    #print(f"Topic Result: {topic_result}\n")
    
    # Step 2a: Provide a detailed explanation
    detailed_explanation_result = detailed_explanation_chain.run({"topic": topic_result, "context": context})
    #print(f"Detailed Explanation Result: {detailed_explanation_result}\n")
    
    # Step 2b: Provide a polished explanation
    polished_language_result = polished_language_chain.run({"topic": topic_result, "context": context})
    #print(f"Polished Language Result: {polished_language_result}\n")
    
    # Ensure we don't use "I don't know" responses
    if "I don't know" in detailed_explanation_result:
        best_result = polished_language_result
    elif "I don't know" in polished_language_result:
        best_result = detailed_explanation_result
    else:
        # Evaluate which detail is better
        best_result = evaluate_results(query, context, detailed_explanation_result, polished_language_result)
    
    #print(f"Best Result: {best_result}\n")
    
    # Step 3: Summarize the best detail
    summary_result = summarize_chain.run({"detail": best_result, "context": context})
    #print(f"Summary Result: {summary_result}\n")
    
    return summary_result

# Example usage
query = query
context = augmented_query

# Run tree of thought with dynamic variables
result = run_tree_of_thought(query, context)
print(result)


Based on the details provided, the key points about the features of Amazon Q Developer are:

- It is a generative AI-powered conversational assistant that can help understand, build, extend, and operate AWS applications by answering questions about AWS architecture, resources, best practices, documentation, and support.

- Key capabilities include:
    - Chat - Answers natural language questions about AWS 
    - Conversation memory - Uses context of conversation to inform future responses
    - Code improvements and advice - Can answer questions about software development and improve/generate code within IDEs
    - Code completion - Automatically generates real-time code suggestions based on existing code and comments
    - Troubleshooting and support - Helps understand errors in AWS console and provides access to live AWS support

- Within IDEs it can chat about code, provide inline code suggestions, generate new code, scan for security vulnerabilities, and upgrade code language versi

# Clean Up Script

In [266]:
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)

In [267]:
bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=kb['knowledgeBaseId'])
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb['knowledgeBaseId'])
oss_client.indices.delete(index=index_name)
aoss_client.delete_collection(id=collection_id)
aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the DeleteDataSource operation: DataSource with id 6C5PDW6EY2 is not found.

In [268]:
delete_iam_role_and_policies()

0

In [269]:
objects = s3_client.list_objects(Bucket=bucket_name)
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'KW363BVTNAVEB1K6',
  'HostId': '43EuEwCZY6RH+R5ZcjYIh+8+sQqcIj+VD+3mDLE8CPfbvynb2uECt9FuCaJBZGhNB7LpVho1h+r6Q32PrAQRCA==',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '43EuEwCZY6RH+R5ZcjYIh+8+sQqcIj+VD+3mDLE8CPfbvynb2uECt9FuCaJBZGhNB7LpVho1h+r6Q32PrAQRCA==',
   'x-amz-request-id': 'KW363BVTNAVEB1K6',
   'date': 'Wed, 05 Jun 2024 18:09:21 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

# Front End

In [3]:
!pip install streamlit pyngrok
!ngrok authtoken 2hU9OKCA3HKDJIr8MfKO23RcZt4_33tVjMrFJsfGQu54RBD42


Authtoken saved to configuration file: /home/ec2-user/.config/ngrok/ngrok.yml                       


In [10]:
with open('app.py', 'w') as f:
    f.write('''
    import streamlit as st

    st.title('Streamlit in Jupyter Example')
    
    name = st.text_input('Enter your name:')
    age = st.slider('Select your age:', 0, 100, 25)
    
    if st.button('Submit'):
        st.write(f'Hello, {name}! You are {age} years old.')
    ''')

In [14]:
from pyngrok import ngrok
import subprocess

# Start the Streamlit app
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Open a ngrok tunnel to the Streamlit app
public_url = ngrok.connect(8501)
print(f'Streamlit app is running at {public_url}')


Streamlit app is running at NgrokTunnel: "https://a75e-34-215-69-55.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8510
  Network URL: http://172.16.62.40:8510
  External URL: http://34.215.69.55:8510

